In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json

In [2]:
import requests as rq
import time

In [7]:
pkgs_to_query = pd.read_csv('./pkgs_to_parse.csv',index_col=0)

## Quering 120 packages per request

In [8]:
fname = './rubygems_pkgs.json'
prefix = 'pkg:gem/' #'pkg:pypi' - more options are available here: https://github.com/package-url/purl-spec

result = []
step=120
pkg = np.full(fill_value=prefix,shape=1,dtype=object)
at = np.full(fill_value='@',shape=1,dtype=object)

counter = 0

for i in tqdm(range(0,len(pkgs_to_query.index),step)):
           
    # changing a step for the last request    
    if i+step > len(pkgs_to_query.index):
        step = len(pkgs_to_query.index)-i
    
    list_of_pkgs = np.array(pkg+np.array(pkgs_to_query.index.tolist()[i:i+step])+at+np.array(pkgs_to_query.values[i:i+step].reshape(-1,)),dtype=str).tolist()

    headers = {'content-type': "application/json",'accept':"application/json"}
    
    data ={'coordinates':list_of_pkgs}
    data_json = json.dumps(data)
    
    req = rq.post(url="https://ossindex.sonatype.org/api/v3/component-report",data=data_json,headers=headers)

    
    # if the status code is 200 - everything went great, otherwise - process the errors
    
    if req.status_code == 200:
        result.extend(req.json())
    
    #processing the errors
    
    if req.status_code == 404 or req.status_code == 400:
        not_found.append(key)
    if req.status_code == 429:
        print('Too many queries')
        while(req.status_code == 429):
            time.sleep(60)
            req = rq.post(url="https://ossindex.sonatype.org/api/v3/component-report",data=data_json,headers=headers)
        if req.status_code == 200:
            result.extend(req.json())
        if req.status_code == 404 or req.status_code == 400:
            not_found.extend(list_of_pkgs)
    
    # if the status code is unknown - there is a problem
    
    if req.status_code == 400 and req.status_code == 200 and req.status_code == 429 and req.status_code == 404:
        print('Something went wrong: ',req.status_code)
    
    time.sleep(31)
    counter += 1
    
    #log the result each 10 steps
    
    if counter == 10:
        with open(fname, 'w') as fout:
            json.dump(result, fout)
        counter = 0

100%|███████████████████████████████████████████████████████████████████████████| 4443/4443 [40:33:11<00:00, 32.86s/it]
